<a href="https://colab.research.google.com/github/asnoldy02-cell/sds510/blob/main/Module_5_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
import pandas as pd
import json
import io

print("Upload jeopardy.json...")
uploaded = files.upload()

filename = next(iter(uploaded))

Upload jeopardy.json...


Saving jeopardy.json to jeopardy (2).json


In [4]:
raw_json = uploaded[filename].decode('utf-8')

In [5]:
try:
    data = json.loads(raw_json)
    df = pd.DataFrame(data)
    print("Loaded JSON as list of objects!")
except:

    df = pd.read_json(io.BytesIO(uploaded[filename]), lines=True)
    print("Loaded JSON as line-delimited JSON!")

df.head()

Loaded JSON as list of objects!


,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680


In [6]:
print("\nPreview of data:")
print(df.head())
print("\nColumns:", df.columns.tolist())


Preview of data:
                          category    air_date  \
0                          HISTORY  2004-12-31   
1  ESPN's TOP 10 ALL-TIME ATHLETES  2004-12-31   
2      EVERYBODY TALKS ABOUT IT...  2004-12-31   
3                 THE COMPANY LINE  2004-12-31   
4              EPITAPHS & TRIBUTES  2004-12-31   

                                            question value       answer  \
0  'For the last 8 years of his life, Galileo was...  $200   Copernicus   
1  'No. 2: 1912 Olympian; football star at Carlis...  $200   Jim Thorpe   
2  'The city of Yuma in this state has a record a...  $200      Arizona   
3  'In 1963, live on "The Art Linkletter Show", t...  $200  McDonald\'s   
4  'Signer of the Dec. of Indep., framer of the C...  $200   John Adams   

       round show_number  
0  Jeopardy!        4680  
1  Jeopardy!        4680  
2  Jeopardy!        4680  
3  Jeopardy!        4680  
4  Jeopardy!        4680  

Columns: ['category', 'air_date', 'question', 'value', 'answer', 'r

In [13]:
import re

def clean_text(text):
    if pd.isnull(text):
        return ""
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [10]:
question_col = None
for col in df.columns:
    if col.lower() == "question":
        question_col = col

if question_col is None:
    raise ValueError("No 'question' or 'Question' column found in JSON.")

df["Question"] = df[question_col].apply(clean_text)

In [16]:
import numpy as np

value_col = None
for col in df.columns:
    if col.lower() == "value":
        value_col = col

if value_col is None:
    raise ValueError("No 'value' or 'Value' column found in JSON.")

def parse_value(val):
    if isinstance(val, str):
        val = val.replace("$", "").replace(",", "")
        if val.lower() == "none" or val == "":
            return np.nan
        try:
            return int(val)
        except:
            return np.nan
    return np.nan

df["ValueNum"] = df[value_col].apply(parse_value)
df = df.dropna(subset=["ValueNum"]).copy()

median_value = df["ValueNum"].median()
df["HighValue"] = (df["ValueNum"] > median_value).astype(int)

print("\nMedian value:", median_value)
print("Class counts:\n", df["HighValue"].value_counts())


Median value: 600.0
Class counts:
 HighValue
0    119805
1     93491
Name: count, dtype: int64


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

X = df["Question"]
y = df["HighValue"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

vectorizer = TfidfVectorizer(
    stop_words="english",
    ngram_range=(1, 2),
    min_df=2
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)



In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

model = MultinomialNB()
model.fit(X_train_vec, y_train)

preds = model.predict(X_test_vec)

accuracy = accuracy_score(y_test, preds)
print("\n========================================")
print("Jeopardy High/Low Value Classifier Report")
print("========================================")
print(f"Accuracy: {accuracy:.4f}\n")
print(classification_report(y_test, preds))


Jeopardy High/Low Value Classifier Report
Accuracy: 0.5895

              precision    recall  f1-score   support

           0       0.59      0.84      0.70     23961
           1       0.57      0.26      0.36     18699

    accuracy                           0.59     42660
   macro avg       0.58      0.55      0.53     42660
weighted avg       0.58      0.59      0.55     42660

